In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 30
BATCH_SIZE = 64

train_transform = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(), transforms.Normalize((0.1307,), (0.3081,))])
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))])
trainset = datasets.FashionMNIST(root = './data/', train = True, download = True, transform = transform)
testset = datasets.FashionMNIST(root = './data/', train = False, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=True)

C:\Users\ryu\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout_p = dropout_p
    
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    

In [5]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [7]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [8]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 1.9346, Accuracy: 62.28%
[2] Test Loss: 1.8104, Accuracy: 66.78%
[3] Test Loss: 1.7724, Accuracy: 70.91%
[4] Test Loss: 1.7187, Accuracy: 76.59%
[5] Test Loss: 1.6980, Accuracy: 77.68%
[6] Test Loss: 1.6885, Accuracy: 78.44%
[7] Test Loss: 1.6837, Accuracy: 78.55%
[8] Test Loss: 1.6789, Accuracy: 78.91%
[9] Test Loss: 1.6746, Accuracy: 79.30%
[10] Test Loss: 1.6715, Accuracy: 79.29%
[11] Test Loss: 1.6685, Accuracy: 79.71%
[12] Test Loss: 1.6671, Accuracy: 79.72%
[13] Test Loss: 1.6656, Accuracy: 79.77%
[14] Test Loss: 1.6644, Accuracy: 80.06%
[15] Test Loss: 1.6614, Accuracy: 80.40%
[16] Test Loss: 1.6599, Accuracy: 80.46%
[17] Test Loss: 1.6596, Accuracy: 80.51%
[18] Test Loss: 1.6574, Accuracy: 80.69%
[19] Test Loss: 1.6564, Accuracy: 80.65%
[20] Test Loss: 1.6554, Accuracy: 80.73%
[21] Test Loss: 1.6544, Accuracy: 80.98%
[22] Test Loss: 1.6538, Accuracy: 80.90%
[23] Test Loss: 1.6526, Accuracy: 81.10%
[24] Test Loss: 1.6523, Accuracy: 81.10%
[25] Test Loss: 1.6524, A